In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json

In [0]:
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

In [0]:
myschema = """
    messageId INT,
    deviceId STRING,
    temperature DOUBLE,
    humidity DOUBLE
"""

IOT_CS="Endpoint=sb://ihsuprodpnres015dednamespace.servicebus.windows.net/;SharedAccessKeyName=iothubowner;SharedAccessKey=ueEkYb3q4+JQJjRuKu5qGgRy0F8My6NmyAIoTCDd/9Q=;EntityPath=iothub-ehub-saniot-55692332-9694222357"
ehConf={
    "eventhubs.connectionString":sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(IOT_CS),
    "eventhubs.consumerGroup":"$Default"
}
eventhub_stream = (
    spark.readStream.format("eventhubs")
        .options(**ehConf)
        .load()
)

df_parsed = eventhub_stream.select(
    from_json(col("body").cast("string"), myschema).alias("data"),
    col("enqueuedTime").alias("EventEnqueuedUtcTime"),
    col("partition").alias("PartitionId"),
    col("systemProperties.iothub-connection-device-id").alias("iotHubDeviceId")
    
)


df_clean = df_parsed.select(
    "data.messageId",
    "data.deviceId",
    "data.temperature",
    "data.humidity",
    "EventEnqueuedUtcTime",
    "PartitionId",
    "iotHubDeviceId"
    
)

df_clean=df_clean.dropDuplicates(["messageId"])

df_clean.writeStream \
    .format("delta") \
    .outputMode("append") \
    .trigger(processingTime="10 seconds") \
    .option("checkpointLocation", "abfss://sink@sanstorage218.dfs.core.windows.net/checkpoint/iotcheckpoint") \
    .option("path", "abfss://sink@sanstorage218.dfs.core.windows.net/data") \
    .start()

In [0]:
%sql
SELECT * FROM DELTA.`abfss://sink@sanstorage218.dfs.core.windows.net/data`



messageId,deviceId,temperature,humidity,EventEnqueuedUtcTime,PartitionId,iotHubDeviceId
13,Raspberry Pi Web Client,27.17663285751035,73.86856374099435,2025-09-28T12:12:57.562Z,0,raspberypiiot
12,Raspberry Pi Web Client,20.234059551400843,69.25804710566875,2025-09-28T12:12:55.515Z,0,raspberypiiot
15,Raspberry Pi Web Client,26.758420598315862,69.18772351341008,2025-09-28T12:13:01.547Z,0,raspberypiiot
10,Raspberry Pi Web Client,27.963014927882778,63.20297100832024,2025-09-28T12:12:51.546Z,0,raspberypiiot
11,Raspberry Pi Web Client,22.905426115583317,68.46230989193907,2025-09-28T12:12:53.546Z,0,raspberypiiot
14,Raspberry Pi Web Client,24.570298372523798,71.6409283439206,2025-09-28T12:12:59.562Z,0,raspberypiiot
6,Raspberry Pi Web Client,31.567889584504673,76.08167115902894,2025-09-28T12:12:43.529Z,0,raspberypiiot
5,Raspberry Pi Web Client,20.520373385152034,77.85377993023901,2025-09-28T12:12:41.544Z,0,raspberypiiot
9,Raspberry Pi Web Client,28.213852029107148,72.92160375298694,2025-09-28T12:12:49.53Z,0,raspberypiiot
8,Raspberry Pi Web Client,20.82815803309068,69.33651839039123,2025-09-28T12:12:47.53Z,0,raspberypiiot
